In [ ]:
from langgraph.prebuilt import ToolNode,tools_condition
from langgraph.graph import StateGraph,START,END,MessagesState
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from typing import TypedDict,List,Annotated

2

In [ ]:
from langchain_core.messages import SystemMessage

SYSTEM_PROMPT = SystemMessage(
    content="""
You are a helpful and knowledgeable AI Travel Agent and Expense Planner.  
Your job is to help users plan trips to any destination worldwide using up-to-date, real-time information from the internet.

For each request, provide a complete, comprehensive, and detailed travel plan. Always offer two options:  
1. An itinerary covering the most popular and iconic tourist attractions.  
2. An alternative itinerary featuring more offbeat, unique, and lesser-known locations in and around the chosen destination.

Your travel plan must include the following details:
- A day-by-day itinerary with activities, timings, and travel tips.
- Recommended hotels or accommodations, including approximate per-night costs and booking tips.
- A list of places to visit with brief descriptions and highlights.
- Suggestions for local restaurants or cafes, with approximate prices and cuisine types.
- Activities and experiences available nearby, with details and any costs.
- Modes of transportation available within the area, including local travel options and approximate fares.
- A detailed cost breakdown, covering accommodation, food, transport, activities, and any other expenses.
- An estimated daily budget to help users plan their finances.
- A short note on the weather condition.

Use the available tools to gather information and make detailed cost breakdowns.

Always provide your response in clear, organized Markdown format for easy reading.

Stay friendly, helpful, and practical — your goal is to make trip planning stress-free, personalized, and cost-effective for every user.
"""
)

## Weather Tool

In [ ]:
import os
import requests

api_key = os.getenv("OPENWEATHER_API_K")
base_url  = "https://api.openweathermap.org/data/2.5"

def get_current_weather(place:str):
    try:
        url = f"{base_url}/weather"
        param = {"q":place,
                "appid":api_key}
        response = requests.get(url=url,params=param)
        return response.json() if response.status_code==200 else {}
    except Exception as e:
        print(f"An error occured while connecting with weather api :{e}")     

def get_forecast_weather(place:str):
    try:
        url = f"{base_url}/forecast"
        param = {"q":place,
                    "appid":api_key,
                    "cnt":10,
                    "units":"metric"}
        response = requests.get(url=url,params=param)
        return response.json() if response.status_code==200 else {}
    except Exception as e:
        print(f"An error occured while connecting with weather api :{e}")

def get_current_weather(city:str)->str:
    """Get current weather for a Given City"""
    weather_data = get_current_weather(city)
    if weather_data:
        temp = weather_data.get('main', {}).get('temp', 'N/A') #Get tempreature of city
        desc = weather_data.get('weather', [{}])[0].get('description', 'N/A') #Get descritption 
        return f"Current weather in {city}: {temp}°C, {desc}"
    return f"Could not fetch weather for {city}"
        
def get_weather_forecast(city:str)->str:
    """Get weather forecast for City"""
    forecast_data = get_forecast_weather(city)
    if forecast_data and "list" in forecast_data:
        forecast_summary  = []
        for i in range(len(forecast_data["list"])):
            item = forecast_data['list'][i]
            date = item['dt_txt'].split(' ')[0]
            temp = item['main']['temp']
            desc = item['weather'][0]['description']
            forecast_summary.append(f"{date}: {temp} degree celcius , {desc}")
        return f"Weather forecast for {city}:\n" + "\n".join(forecast_summary)
    return f"Could not fetch forecast for {city}"

In [ ]:
llm = ChatOpenAI(model="gpt-4.1-nano")
tools = []
llm_with_tools = llm.bind_tools(tools)

def agent_function(state:MessagesState):
    return {"messages":[llm_with_tools.invoke(state["messages"])]}

In [ ]:
graph_builder = StateGraph(MessagesState)
graph_builder.add_node("agent",agent_function)
graph_builder.add_edge(START,"agent")
graph_builder.add_node("tools",ToolNode(tools=tools))
graph_builder.add_conditional_edges("agent",tools_condition)
graph_builder.add_edge("tools","agent")
graph_builder.add_edge("agent",END)

graph = graph_builder.compile()

from IPython.display import display,Image
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
graph.invoke({"messages":"what is the weather of Islamabad?"})